In [3]:
import time
from datetime import date
import pandas as pd
import os
import glob
import pycytominer
import sys
import CBE_utils as CBE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import gc

In [5]:
import importlib
importlib.reload(CBE)

<module 'CBE_utils' from '/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/notebooks_revision/CBE_utils.py'>

In [6]:
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/output/"

annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"
feature_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/reduced_features/"

In [7]:
def list_folders(directory):
    try:
        # List all entries in the specified directory
        entries = os.listdir(directory)
        
        # Filter out non-folder entries
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        return folders
    
    except FileNotFoundError:
        
        return f"The directory '{directory}' does not exist."
    
    except PermissionError:
        
        return f"Permission denied to access the directory '{directory}'."

In [8]:
# TODO: Check how many unique plates exist
source_list = list_folders(input_path)

files = []

for source in source_list:
    
    print(source)

    source_path = os.path.join(input_path, source)

    plate_list = list_folders(source_path)

    

    for plate in plate_list:
        
        plate_path = os.path.join(source_path, plate)

        

        raw_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4].csv"
        if glob.glob(plate_path + os.sep + raw_pattern , recursive=True):

            raw_file_exists = True

        else:

            raw_file_exists = False

        norm_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4]_mad_robustize.csv"

        if glob.glob(plate_path + os.sep + norm_pattern  , recursive=True):

            norm_file_exits = True
            
        else:

            norm_file_exits = False
       
        processed_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4]_mad_robustize_reduced-corr.csv"
        
        if glob.glob(plate_path + os.sep + processed_pattern , recursive=True):

            reduced_file_exists = True
            
        else:

            reduced_file_exists = False
       
        files.append({'imaging_site': source, 
                     'plate': plate, 
                     'raw_file': raw_file_exists, 
                     'norm_file': norm_file_exits, 
                     'reduced_file': reduced_file_exists})

IMTM
USC
FMP
MEDINA


In [9]:
file_check = pd.DataFrame(files)

In [10]:
file_check['batch'], file_check['plate_name'], file_check['replicate'] = file_check['plate'].str.split('_', n=2).str

<ipython-input-10-50efb533959c>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  file_check['batch'], file_check['plate_name'], file_check['replicate'] = file_check['plate'].str.split('_', n=2).str


In [11]:
file_check

,imaging_site,plate,raw_file,norm_file,reduced_file,batch,plate_name,replicate
0,IMTM,2024-09-05_C1269_R2,True,True,True,2024-09-05,C1269,R2
1,IMTM,2025-01-09_C1242_R2,True,True,True,2025-01-09,C1242,R2
2,IMTM,2024-11-06_C1288_R1,True,True,True,2024-11-06,C1288,R1
3,IMTM,2024-06-25_C1240_R4,True,True,True,2024-06-25,C1240,R4
4,IMTM,2025-01-13_C1242_R3,True,True,True,2025-01-13,C1242,R3
...,...,...,...,...,...,...,...,...
959,MEDINA,2024-10-28_C1041_R3,True,True,True,2024-10-28,C1041,R3
960,MEDINA,2025-02-10_C1065_R4,True,True,True,2025-02-10,C1065,R4
961,MEDINA,2025-01-23_C1066_R3,True,True,True,2025-01-23,C1066,R3
962,MEDINA,2024-09-25_C1038_R2,True,True,True,2024-09-25,C1038,R2


In [28]:
def file_checker(file_check_dataframe, site_string):

    site_files = file_check_dataframe[file_check_dataframe['imaging_site'] == site_string] 

    print( site_string + ' file check:')
    print(str(site_files['raw_file'].count()) + ' raw file(s)')
    print(str(site_files['norm_file'].count()) + ' normalized file(s)')
    print(str(site_files['reduced_file'].count()) + ' feature reduced file(s)')

In [29]:
file_checker(file_check, 'IMTM')

IMTM file check:
247 raw file(s)
247 normalized file(s)
247 feature reduced file(s)


In [30]:
file_checker(file_check, 'FMP')

FMP file check:
184 raw file(s)
184 normalized file(s)
184 feature reduced file(s)


In [31]:
file_checker(file_check, 'USC')

USC file check:
283 raw file(s)
283 normalized file(s)
283 feature reduced file(s)


In [32]:
file_checker(file_check, 'MEDINA')

MEDINA file check:
250 raw file(s)
250 normalized file(s)
250 feature reduced file(s)
